In [ ]:
from sentence_transformers import SentenceTransformer                      # 문장 임베딩 생성을 위한 Transformer 모델
import psycopg2                                                        # PostgreSQL 데이터베이스 연결 라이브러리
from pgvector.psycopg2 import register_vector                          # pgvector 확장을 위한 벡터 타입 등록

# DB 접속 정보 (본인 환경에 맞게 수정)
DB_NAME = "postgres"                                                   # 데이터베이스 이름
DB_USER = "postgres"                                                   # 데이터베이스 사용자명
DB_PASSWORD = "0430"                                                   # 데이터베이스 비밀번호
DB_HOST = "localhost"                                                  # 데이터베이스 호스트 주소

# DB 연결
conn = psycopg2.connect(dbname="postgres", user="postgres", password="0430", host="localhost")  # PostgreSQL 연결 객체 생성
cur = conn.cursor()                                                    # 쿼리 실행을 위한 커서 객체 생성

# pgvector 타입 등록
register_vector(conn)                                                  # PostgreSQL에서 vector 타입 사용 가능하도록 등록

In [ ]:
# 모델 로드
model = SentenceTransformer('sentence-transformers/paraphrase-MiniLM-L6-v2')  # 384차원 임베딩을 생성하는 다국어 모델 로드

# 리뷰 데이터
reviews = [                                                            # 테스트용 샘플 리뷰 데이터 리스트
    "배송이 빠르고 제품도 좋아요.",                                        # 긍정적 배송/품질 리뷰
    "품질이 기대 이상입니다!",                                           # 품질 만족 리뷰
    "생각보다 배송이 오래 걸렸어요.",                                      # 배송 지연 불만 리뷰
    "배송은 느렸지만 포장은 안전했어요.",                                   # 배송/포장 혼합 평가 리뷰
    "아주 만족스러운 제품입니다."                                         # 전반적 만족 리뷰
]

# 리뷰 임베딩 생성
embeddings = model.encode(reviews)                                     # 리뷰 텍스트를 384차원 벡터로 변환 (numpy array 반환)

In [ ]:
cur.execute("CREATE EXTENSION IF NOT EXISTS vector;")                 # pgvector 확장 생성 (벡터 연산을 위한 PostgreSQL 확장)
conn.commit()                                                          # 변경사항을 데이터베이스에 반영

cur.execute("""                                                        
CREATE TABLE IF NOT EXISTS review_vectors (
    id SERIAL PRIMARY KEY,                                             
    review TEXT,                                                       
    embedding VECTOR(384)                                              
);
""")
conn.commit()                                                          # 테이블 생성 쿼리 실행 결과 커밋

In [ ]:
insert_sql = "INSERT INTO review_vectors (review, embedding) VALUES (%s, %s)"  # 리뷰와 임베딩을 테이블에 삽입하는 SQL 쿼리
for review, embedding in zip(reviews, embeddings):                     # 리뷰 텍스트와 해당 임베딩을 순회
    cur.execute(insert_sql, (review, embedding.tolist()))             # numpy array를 list로 변환하여 DB에 저장
conn.commit()                                                          # 모든 데이터 삽입 작업을 데이터베이스에 최종 반영

In [ ]:
# 모델 로딩
model = SentenceTransformer('sentence-transformers/paraphrase-MiniLM-L6-v2')  # 동일한 모델을 다시 로드 (일관성 유지)

reviews = [                                                            # 추가 데이터 삽입용 리뷰 리스트 (중복 데이터)
    "배송이 빠르고 제품도 좋아요.",                                        # 긍정적 배송/품질 리뷰
    "품질이 기대 이상입니다!",                                           # 품질 만족 리뷰  
    "생각보다 배송이 오래 걸렸어요.",                                      # 배송 지연 불만 리뷰
    "배송은 느렸지만 포장은 안전했어요.",                                   # 배송/포장 혼합 평가 리뷰
    "아주 만족스러운 제품입니다."                                         # 전반적 만족 리뷰
]

insert_sql = "INSERT INTO review_vectors (review, embedding) VALUES (%s, %s)"  # 데이터 삽입 SQL 쿼리 정의
for review in reviews:                                                 # 각 리뷰에 대해 개별적으로 처리
    embedding = model.encode(review).tolist()                          # 리뷰를 실시간으로 임베딩하고 list로 변환
    cur.execute(insert_sql, (review, embedding))                       # 리뷰와 임베딩을 데이터베이스에 삽입
conn.commit()                                                          # 모든 삽입 작업을 데이터베이스에 반영

In [6]:
from pgvector import Vector

query_text = "배송이 느렸어요"
query_vec = Vector(model.encode(query_text).tolist())

select_sql =  """
    SELECT review, embedding, (1 - (embedding <=> %s::vector)) AS similarity
    FROM review_vectors
    ORDER BY similarity DESC
    LIMIT 3;
"""

cur.execute(select_sql, (query_vec,))
results = cur.fetchall()


In [7]:
cur.close()
conn.close()


In [8]:
# 결과 확인
print("쿼리:", query_text)
print("유사한 리뷰:")
print("-" * 50)
print(f"{'리뷰':<50} | {'유사도':<10}")
print("-" * 50)
print("\n".join(f"{review[:50]:<50} | {similarity:.4f}" for review, _, similarity in results))  


쿼리: 배송이 느렸어요
유사한 리뷰:
--------------------------------------------------
리뷰                                                 | 유사도       
--------------------------------------------------
내구성이 약해 쉽게 부서졌습니다.                                 | 0.9417
사이즈가 맞지 않아 반품했습니다.                                 | 0.9356
배송 상태가 엉망이었습니다.                                    | 0.9301
